Complete the following tasks with Python Boto code
1. Create ECS Cluster with 1 Container Instance
2. Create a Task Definition (apache and wordpress)
3. Create an ELB and Target Group to later associate with the ECS Service
4. Create a Service that runs the Task Definition
5. Confirm Everything is Working
6. Scale Up the Service to 4 Tasks.
7. Clean It All Up

Docs:
https://docs.aws.amazon.com/AmazonECS/latest/developerguide/launch_container_instance.html

In [ ]:
import boto3
import pprint
import time

ecs_client = boto3.client('ecs')

### Creating our cluster

In [ ]:
mycluster = 'myTestCluster'

createClusterResponse = ecs_client.create_cluster(
                        
                        clusterName = mycluster,

                        )



In [ ]:
pprint.pprint(createClusterResponse)

In [ ]:
ec2_client = boto3.client('ec2')

user_data_script ="""#!/bin/bash
echo ECS_CLUSTER=myTestCluster >> /etc/ecs/ecs.config"""

ec2Response = ec2_client.run_instances(

                ImageId = 'ami-0b9a214f40c38d5eb', #amazon-ecs-optimized image
#                 ImageId = 'ami-00129b193dc81bc31',
                MinCount = 1,
                MaxCount = 4,
                InstanceType = 't2.micro',
                KeyName='XXXXX',
                SecurityGroupIds=['sg-XXXXXXXXXXX'],
                IamInstanceProfile = {
                    "Name" : "ecsInstanceRole"
                },
                UserData = user_data_script,
                SubnetId='subnet-XXXXXX',

            )

In [ ]:
pprint.pprint(ec2Response)

Save the ids for later use while cleaning up

In [ ]:
# pprint.pprint(ec2Response['Instances'][1]['InstanceId'])
ids =[]
for i in ec2Response['Instances']:
    ids.append(i['InstanceId'])
    
print(ids)

## Creating a target group and load balancer.

In [ ]:
mytargetgroup = 'myTG'
tg_response = boto3.client('elbv2').create_target_group(
    Name=mytargetgroup,
    Port=80,
    Protocol='HTTP',
    VpcId='vpc-XXXXXXX',
)

elbname = 'myElb'
elb_response = boto3.client('elbv2').create_load_balancer(

                Name = elbname,
#                 Listeners = [
#                   {
#                     'InstancePort': 8080,
#                     'InstanceProtocol': 'HTTP',
#                     'LoadBalancerPort': 80,
#                     'Protocol': 'HTTP',
#                   },      
#                 ],
                Subnets=[
                    'subnet-XXXXXXXX',
                    'subnet-XXXXXXXX',
                    'subnet-XXXXXXXX',
                    'subnet-XXXXXXXX',
                    'subnet-XXXXXXXX',
                    'subnet-XXXXXXXX',
                    'subnet-XXXXXXXX',
                ],
                SecurityGroups=[
                    'sg-XXXXXX',
                ],
                )

pprint.pprint(tg_response)
pprint.pprint(elb_response)


In [ ]:
# pprint.pprint(elb_response)
tgarn = tg_response['TargetGroups'][0]['TargetGroupArn']
lbarn = elb_response['LoadBalancers'][0]['LoadBalancerArn']

## Creating a listner for the load balancer which associates the loadbalancer with our target group

In [ ]:
listner_response =  boto3.client('elbv2').create_listener(
    DefaultActions=[
        {
            'TargetGroupArn': tgarn,
            'Type': 'forward',
        },
    ],
    LoadBalancerArn=lbarn,
    Port=80,
    Protocol='HTTP',
)


In [ ]:
pprint.pprint(listner_response)

## Creating the task definition

In [ ]:
task_response = ecs_client.register_task_definition(
    
        containerDefinitions= [
        {
            "environment": [
                {
                    "name": "WORDPRESS_DB_PASSWORD",
                    "value": "ChangeMeIfYouWant"
                }
            ],
            "essential": True,
            "image": "wordpress",
#             "image": "wordpress:apache",
#             "image": "wordpress:4.9.8-apache",
            "memory": 300,
            "cpu": 10,
            "links": [
                "my-wpdb:mysql"
            ],
#             "mountPoints": [
#                 {
#                     "containerPath": "/var/www/html",
#                     "sourceVolume": "vol-012b57b135a18d576"
#                 }
#             ],
            "name": "my-wp",
            "portMappings": [
                {
                    "containerPort": 80,
                    "hostPort": 8080
                }
            ]
        },
        {
            "environment": [
                {
                    "name": "MYSQL_ROOT_PASSWORD",
                    "value": "ChangeMeIfYouWant"
                }
            ],
            "essential": True,
            "image": "mariadb",
            "name": "my-wpdb",
            "memory": 300,
            "cpu": 10,
            "portMappings": [
                {
                    "containerPort": 3306,
                    "hostPort": 8081
                }
            ]
        }
    ],

#     volumes= [
#         {
#             "host": {
#                 "sourcePath": "./"
#             },
#             "name": "vol-XXXXXX"
#         }
#     ],
        family="wptry"
    )


pprint.pprint(task_response)

task_arn = task_response['taskDefinition']['taskDefinitionArn']

## Launching a service based on the above task definiton

In [ ]:
myservice = 'wordpress_service'

create_service_response = ecs_client.create_service(
                
                            cluster = mycluster,
                            serviceName= myservice,
                            taskDefinition= task_arn,
                            desiredCount=1,
                            clientToken= 'my_token',
                            launchType='EC2',
                            loadBalancers=[
                              {
                                'targetGroupArn': tgarn,
#                                 'loadBalancerName':elbname,
                                'containerPort': 80,
                                'containerName':'my-wp'
                              },  
                            ],
                            healthCheckGracePeriodSeconds=500,
                        )

pprint.pprint(create_service_response)

## Scaling up the service to 4

In [ ]:
boto3.client('ecs').update_service(
    cluster=mycluster,
    service=myservice,
    desiredCount=4,
)

## Deleting all the AWS Resources

In [ ]:
boto3.client('ecs').update_service(
    cluster=mycluster,
    service=myservice,
    desiredCount=0,
)
boto3.client('ecs').delete_service(
    cluster=mycluster,
    service=myservice,
    force=True,
)

boto3.client('elbv2').delete_load_balancer(
    LoadBalancerArn = lbarn,
)



In [ ]:
time.sleep(10)
boto3.client('elbv2').delete_target_group(
    TargetGroupArn=tgarn,
)

In [ ]:
boto3.resource('ec2').instances.filter(InstanceIds=ids).stop()
boto3.resource('ec2').instances.filter(InstanceIds=ids).terminate()

In [ ]:
time.sleep(30)
boto3.client('ecs').delete_cluster(
    cluster = mycluster,
)